In [1]:
# import packages
! pip install openai
! pip install python-dotenv


^C



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import libraries 
import openai
import os
import json
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [3]:
# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key from environment variable
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = f"https://{os.getenv('OPENAI_SERVICE_NAME')}.openai.azure.com/"
openai.api_version = os.getenv("OPENAI_API_VERSION")

# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'

# Output embeddings to docVectors.json file
with open("../output/docVectors.json", "w") as f:
    json.dump(input_data, f)


In [8]:
# Generate a query embedding
response = openai.Embedding.create(
    input="what is Azure Search",
    engine="text-embedding-ada-002"
)
embedding = response['data'][0]['embedding']

# Output embeddings to queryVector.json file
with open("../output/queryVector.json", "w") as f:
    json.dump(embedding, f)

print(embedding)

[-0.003434219863265753, -0.01066913828253746, 0.031920041888952255, 0.001425868715159595, -1.0712976290960796e-05, -0.026619451120495796, -0.0012007633922621608, 0.030832741409540176, -0.0009234775207005441, 0.01902776211500168, -0.008348915725946426, -0.011843811720609665, 0.03353157639503479, 0.00611121254041791, -0.0035701324231922626, 0.03357040882110596, -0.02028980851173401, -0.013057316653430462, -0.003004638943821192, -0.02494966797530651, -0.02007623016834259, -0.032347194850444794, -0.02176542952656746, 0.04442400112748146, -0.023726454004645348, -0.008407164365053177, 0.006912125740200281, 0.009586691856384277, -0.004002140369266272, 0.008523660711944103, 0.030172593891620636, -0.03106573410332203, 0.029337702319025993, 0.039356403052806854, -0.016561919823288918, -0.013134980574250221, -0.00997986737638712, -0.005203510634601116, -0.0013372828252613544, -0.014562062919139862, 0.005548146087676287, 0.0060529643669724464, -0.030755076557397842, 0.0033953876700252295, 0.011698